Caustics ([wikipedia](https://en.wikipedia.org/wiki/Caustic_(optics)) are luminous patterns which are resulting from the superposition of smoothly deviated light rays. It is for instance the heart-shaped pattern in your cup of coffee which is formed as the rays of the sun are reflected on the cup's surface. It is also the wiggly pattern of light curves that you will see on the floor of a pool as the sun's light is *refracted* at the surface of the water. Here, we simulate that particular physical phenomenon. Simply because they are mesmerizingly beautiful, but also as it is of interest in visual neuroscience. Indeed, it speaks to how images are formed (more on this later), hence how the brain may understand images. 

In this post, I will develop a simple formalism to generate such patterns, with the paradoxical result that it is *very* simple to code yet generates patterns with great complexity, such as: 
<BR>
<center><img src="https://laurentperrinet.github.io/sciblog/files/2020-06-19_caustique/2020-06-19_caustique.gif" width=61.8%/> </center>
<BR>

This is joint work with artist [Etienne Rey](https://laurentperrinet.github.io/authors/etienne-rey/), in which I especially follow the ideas put forward in the series [Turbulence](http://ondesparalleles.org/projets/turbulences/).
    

[![DOI](https://zenodo.org/badge/273226625.svg)](https://zenodo.org/badge/latestdoi/273226625)



<!-- TEASER_END -->

Let's begin with an import of the libraries that we will need: [numpy](https://numpy.org/) and [matplotlib](https://matplotlib.org/) (essentials!) but also [MotionClouds](https://github.com/NeuralEnsemble/MotionClouds) to generate a simple waveform (more on this later in this post). These are all installable using [pip](https://pip.pypa.io/en/stable/) (see the [requirements.txt](https://github.com/NaturalPatterns/2020_caustiques/blob/master/requirements.txt) file).

In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import MotionClouds as mc

Let's define an unique identifier

In [2]:
import datetime
date = datetime.datetime.now().date().isoformat() # https://en.wikipedia.org/wiki/ISO_8601
tag = f'{date}_caustique'
tag = '2020-06-19_caustique' # overwriting
print(f'Tagging our simulations with tag={tag}')

Tagging our simulations with tag=2020-06-19_caustique


And a `Namespace` object to store all parameters:

In [3]:
def init(args=[]):
    import argparse

    parser = argparse.ArgumentParser()
    parser.add_argument("--tag", type=str, default='caustique', help="Tag")
    parser.add_argument("--figpath", type=str, default='2020-06-19_caustique', help="Folder to store images")
    parser.add_argument("--nx", type=int, default=5*2**8, help="number of pixels (vertical)")
    parser.add_argument("--ny", type=int, default=8*2**8, help="number of pixels (horizontal)")
    parser.add_argument("--bin_dens", type=int, default=4, help="relative bin density")
    parser.add_argument("--nframe", type=int, default=2**7, help="number of frames")
    parser.add_argument("--seed", type=int, default=42, help="seed for RNG")
    parser.add_argument("--H", type=float, default=20., help="depth of the pool")
    parser.add_argument("--sf_0", type=float, default=0.004, help="sf")
    parser.add_argument("--B_sf", type=float, default=0.002, help="bandwidth in sf")
    parser.add_argument("--V_Y", type=float, default=0.3, help="horizontal speed")
    parser.add_argument("--V_X", type=float, default=0.3, help="vertical speed")
    parser.add_argument("--B_V", type=float, default=2.0, help="bandwidth in speed")
    parser.add_argument("--theta", type=float, default=2*np.pi*(2-1.61803), help="angle with the horizontal")
    parser.add_argument("--B_theta", type=float, default=np.pi/12, help="bandwidth in theta")
    parser.add_argument("--fps", type=float, default=18, help="bandwidth in theta")
    parser.add_argument("--verbose", type=bool, default=False, help="Displays more verbose output.")

    opt = parser.parse_args(args=args)

    if opt.verbose:
        print(opt)
    return opt
opt = init()
opt.tag = tag

In [4]:
opt

Namespace(B_V=2.0, B_sf=0.002, B_theta=0.2617993877991494, H=20.0, V_X=0.3, V_Y=0.3, bin_dens=4, figpath='2020-06-19_caustique', fps=18, nframe=128, nx=1280, ny=2048, seed=42, sf_0=0.004, tag='2020-06-19_caustique', theta=2.399988291783386, verbose=False)

This object is always handy when we will try to explore some parameters.

A last utility function is a way to convert the resulting numpy array to a set of images and then to an animated GIF using [imageio](https://imageio.github.io/) and [pygifsicle](https://github.com/LucaCappelletti94/pygifsicle):

In [5]:
def make_gif(gifname, fnames, fps):
    import imageio

    with imageio.get_writer(gifname, mode='I', fps=fps) as writer:
        for fname in fnames:
            writer.append_data(imageio.imread(fname))

    from pygifsicle import optimize
    optimize(str(gifname))
    return gifname

In [5]:
from IPython.display import Image, display
width = 1024

# What is a caustic?

A caustic usually refers to the image which is formed by the convergence or divergence of rays on a surface. Mathematically, this may be understood as the *enveloppe* of a set of rays, and intuitively, it is best understood by looking at the best known example: When the sun hits a glass or cup and you see heart-shaped pattern (actually more a kidney than a heart, hence the name "nephroid"):

<BR>
<center><img src="https://upload.wikimedia.org/wikipedia/commons/7/7e/Brennlinie.GIF" width=38.2%/> </center>
<BR>

(image by [User:Paul venter](https://commons.wikimedia.org/wiki/User:Paul_venter)).

Their study is essential in optics, as these are the bases for lenses and therefore of central importance in our image-centered society :-)

They are also of interest simply because they create mesmerizing patterns in nature, for instance the pattern of wiggly lines on swimming pool bottoms, or on the sea floor at shallow depths. 

The formation of that specific pattern is very well illustrated by this animation (credit [Jacopo Bertolotti](https://twitter.com/j_bertolotti), check-out that twitter [thread](https://twitter.com/j_bertolotti/status/1229743952594640897)) where light comes from  the bottom, travels through air, hits the surface and gets deviated:

<BR>
<center><img src="https://upload.wikimedia.org/wikipedia/commons/2/2a/Caustics.gif" width=61.8%/> </center>
<BR>

What happens? Light travels fast (*very* fast) but it will travel slower in water than in the air. A consequence of this is that a ray changes slightly its direction as it hits the water: the light ray is *refracted*. That is why a pen which is half immerged looks "as if" it is broken. In fact, it's only its image that gets distorted:
    
<BR>
<center><a title="Ulflund / CC0" href="https://commons.wikimedia.org/wiki/File:Pen_in_water.jpg"><img width="512" alt="Pen in water" src="https://upload.wikimedia.org/wikipedia/commons/thumb/a/ab/Pen_in_water.jpg/512px-Pen_in_water.jpg"></a></center>
<BR>

The laws governing this deviations are well known as [Snell's law](https://en.wikipedia.org/wiki/Snell%27s_law) and state that

$$
\frac{\sin\theta_2}{\sin\theta_1} = \frac{n_1}{n_2}
$$
Where $n_1\approx 1$ and  $n_2\approx1.33$ are the refractive indices of air and water, respectively. (Note that, as a consequence, the light in the water is always slightly closer to the normal of the surface.)

<BR>
<center>
<a title="Cristan at English Wikipedia, Sawims at Dutch Wikipedia / Public domain" href="https://commons.wikimedia.org/wiki/File:Snells_law.svg"><img width="512" alt="Snells law" src="https://upload.wikimedia.org/wikipedia/commons/thumb/d/d1/Snells_law.svg/512px-Snells_law.svg.png"></a>        
</center>
<BR>
    
Going back to the swimming pool, considering that the sun is emitting an uniform pattern of parallel rays from an incident angle $\theta_I$, we may apply this law to all of these light rays. The important ingredient is that the surface of the water (given as the surface height $z(x, y)$) changes smoothly and will deviate the light rays non-uniformly, thus creating the patterns. Writing down the equation, the displacement resulting from the angle of refraction is given by:

$$
\Delta x(x, y) = H \cdot \arcsin [ \frac 1 n \sin( \theta_I - \arctan(\frac{\partial}{\partial x} z(x, y)) + \arctan(\frac{\partial}{\partial x} z(x, y) ]
$$

Where $H$ is the pool's depth, $n$ is the refractive index of water (that of air being set to $1$) and $\theta_I$ the incident angle.
The same formula applies in the other spatial dimension $y$ by computing the angle of the surface along this dimension as $\arctan(\frac{\partial}{\partial y} z(x, y))$. 

In the limit of a zenithal light source (that is, the sun at the vertical of the pool) and in the limit of small angles, the equations simplifies to:
$$
\Delta x(x, y) \approx H' \cdot \frac{\partial z(x, y)}{\partial x}
$$
and
$$
\Delta y(x, y) \approx H' \cdot \frac{\partial z(x, y)}{\partial y}
$$

with $ H' = \frac{n-1}{n} H$. What is interesting here is that we will transform an incoming "image" (here the uniform light source from the sun) by a set of small and local deformations of the support of the image. The equations are ultra-simple, but the consequences will paradoxically look complex.
Thinking about images as densities which we may discretize on a rectangular grid (in the `__init__` function), it is very easy to generate a wave pattern ass a smooth surface in space and time (in the `wave` function), simulate the transformation on that surface (in the `transform` function) and compute the resulting image as an histogram  (in the `plot` function):

In [6]:
class Caustique:
    def __init__(self, opt):
        """
        Image coordinates follow 'ij' indexing, that is, 
        * their origin at the top left, 
        * the X axis is vertical and goes "down",
        * the Y axis is horizontal and goes "right".
        
        """
        self.ratio = opt.ny/opt.nx # ratio between height and width (>1 for portrait, <1 for landscape)
        X = np.linspace(0, 1, opt.nx, endpoint=False) # vertical
        Y = np.linspace(0, self.ratio, opt.ny, endpoint=False) # horizontal
        self.xv, self.yv = np.meshgrid(X, Y, indexing='ij')
        self.opt = opt
        # https://stackoverflow.com/questions/16878315/what-is-the-right-way-to-treat-python-argparse-namespace-as-a-dictionary
        self.d = vars(opt)

    def wave(self):
        # A simplistic model of a wave using https://github.com/NeuralEnsemble/MotionClouds
        import MotionClouds as mc
        fx, fy, ft = mc.get_grids(self.opt.nx, self.opt.ny, self.opt.nframe)
        env = mc.envelope_gabor(fx, fy, ft, V_X=self.opt.V_Y, V_Y=self.opt.V_X, B_V=self.opt.B_V,
                                sf_0=self.opt.sf_0, B_sf=self.opt.B_sf, 
                                theta=self.opt.theta, B_theta=self.opt.B_theta)
        z = mc.rectif(mc.random_cloud(env, seed=self.opt.seed))
        return z

    def transform(self, z_):
        xv, yv = self.xv.copy(), self.yv.copy()
        
        dzdx = z_ - np.roll(z_, 1, axis=0)
        dzdy = z_ - np.roll(z_, 1, axis=1)
        xv = xv + self.opt.H * dzdx
        yv = yv + self.opt.H * dzdy

        xv = np.mod(xv, 1)
        yv = np.mod(yv, self.ratio)

        return xv, yv

    def plot(self, z, gifname=None, dpi=150):
        if gifname is None:
            os.makedirs(self.opt.figpath, exist_ok=True)
            gifname=f'{self.opt.figpath}/{self.opt.tag}.gif'
        binsx, binsy = self.opt.nx//self.opt.bin_dens, self.opt.ny//self.opt.bin_dens

        hist = np.zeros((binsx, binsy, self.opt.nframe))
        for i_frame in range(self.opt.nframe):
            xv, yv = self.transform(z[:, :, i_frame])
            hist[:, :, i_frame], edge_x, edge_y = np.histogram2d(xv.ravel(), yv.ravel(), 
                                                                 bins=[binsx, binsy], 
                                                                 range=[[0, 1], [0, self.ratio]], 
                                                                 density=True)

        hist /= hist.max()
        subplotpars = matplotlib.figure.SubplotParams(left=0., right=1., bottom=0., top=1., wspace=0., hspace=0.,)
        fnames = []
        for i_frame in range(self.opt.nframe):
            fig, ax = plt.subplots(figsize=(binsy/dpi, binsx/dpi), subplotpars=subplotpars)
            ax.pcolormesh(edge_y, edge_x, hist[:, :, i_frame], vmin=0, vmax=1, cmap=plt.cm.Blues_r)
            fname = f'/tmp/{self.opt.tag}_frame_{i_frame}.png'
            fig.savefig(fname, dpi=dpi)
            fnames.append(fname)
            plt.close('all')

        return make_gif(gifname, fnames, fps=self.opt.fps)


# a first instance

Let's generate a first instance of this caustic with the default parameters. Let's first have a look at the (simplistic) waveform we are using:

In [7]:
if not os.path.isfile(f'{opt.figpath}/{opt.tag}{mc.vext}'):
    print('Doing', f'{opt.figpath}/{opt.tag}.{mc.vext}')
    c = Caustique(opt)
    z = c.wave()
    mc.anim_save(z.swapaxes(0, 1), f'{opt.figpath}/{opt.tag}')
mc.in_show_video(f'{opt.tag}', figpath=opt.figpath)

""


It has many desirable properties: it is smooth in time and space, it is stochastic and periodic (no border effect). As such we may apply our simple transform and as a result we get:

In [8]:
gifname = f'{opt.figpath}/{opt.tag}.gif'
if not os.path.isfile(gifname):
    c = Caustique(opt)
    z = c.wave()
    gifname = c.plot(z)
display(Image(url=gifname, width=width))

This image ressembles the wiggling lines that we expected to find, great! Some properties:

* it is periodic in space and time by construction which may look unrealistic for a pool, but looks better as you do not have border effects...
* there are different classes of visual objects: veil-like smooth curves, traveling waves with a difference in their group/phase speed, nephroid-like intersection points, smooth gradients, ... which mainly depend on the surface's ondulation,
* when looking at this video long enough (I recommend opening this animated GIF in a separate tab and to zoom on it fullscreen), you may perceive some three-dimensionnal percepts, especially when some features change their speed..

## exploring parameters
Is there more to see in our system?
The first thing to do is to explore is the effect of the pool's depth:

In [9]:
N_scan = 9
base = 2

In [10]:
opt = init()
opt.tag = tag

c = Caustique(opt)
z = None
for H_ in c.opt.H*np.logspace(-1, 1, N_scan, base=base):
    print(f'H = {H_:.3f}')
    c.opt.H = H_
    gifname = f'{opt.figpath}/{opt.tag}_H_{H_:.3f}.gif'
    if not os.path.isfile(gifname):
        if z is None:
            z = c.wave()
        url=c.plot(z, gifname=gifname)
    display(Image(url=gifname, width=width))

H = 10.000


H = 11.892


H = 14.142


H = 16.818


H = 20.000


H = 23.784


H = 28.284


H = 33.636


H = 40.000


This shows that for this waveform, there is an optimal depth to generate more concentrated caustics. This corresponds to the depth where the curvature of the waveform concentrates light as a lens. Note that at great distortions some ripples appear which are due to our sampling of space.

We may now explore the parameters of this waveform to see their effect on the resulting caustic:

In [11]:
opt = init()

for variable in ['sf_0', 'B_sf', 'theta', 'B_theta', 'V_X', 'V_Y', 'B_V']:
    print(f'======{variable}======')
    for modul in np.logspace(-1, 1, N_scan, base=base):
        opt = init()
        opt.tag = tag

        c = Caustique(opt)
        c.d[variable] *= modul

        print(f'{variable}={variable}(default)*{modul:.3f}={c.d[variable]:.3E}')
        gifname = f'{opt.figpath}/{opt.tag}_{variable}_modul_{modul:.3f}.gif'
        if not os.path.isfile(gifname):
            print('Doing ', gifname)
            z = c.wave()
            mcname = f'{opt.figpath}/{opt.tag}_{variable}_modul_{modul:.3f}'
            if not os.path.isfile(f'{mcname}{mc.vext}'): 
                print('Doing ', f'{mcname}{mc.vext}')
                mc.anim_save(z.swapaxes(0, 1), f'{mcname}')
            url=c.plot(z, gifname=gifname)
        display(Image(url=gifname, width=width))

======sf_0======
sf_0=sf_0(default)*0.500=2.000E-03


sf_0=sf_0(default)*0.595=2.378E-03


sf_0=sf_0(default)*0.707=2.828E-03


sf_0=sf_0(default)*0.841=3.364E-03


sf_0=sf_0(default)*1.000=4.000E-03


...

B_V=B_V(default)*1.000=2.000E+00


B_V=B_V(default)*1.189=2.378E+00


B_V=B_V(default)*1.414=2.828E+00


B_V=B_V(default)*1.682=3.364E+00


B_V=B_V(default)*2.000=4.000E+00


Future developments should:

* check if the approximation of small angles qualitatively changes the resulting images (should not, what maters is the transport of the density, not it's precision),
* use a more realistic waveform such as in https://laurentperrinet.github.io/sciblog/posts/2016-04-24_a-wave-going-backwards.html
* use different indices for different wavelengths (+add gamma correction for rendering),
* make it live (using pyglet for instance)

This development happens in https://github.com/NaturalPatterns/2020_caustiques - I am happy to welcome any people that would be interested in exploring this.

This shows that for this waveform, there is an optimal depth to generate more concentrated caustics. This corresponds to the depth for which the curvature of the waveform concentrates light as a lens. 
